In [6]:
import numpy as np
import pandas as pd
import re
import json
import requests
import time
import xml.etree.ElementTree as ET
#!pip install folium
import folium
import os
import zipfile
from sklearn.cluster import DBSCAN
#!pip install geopy
from geopy.distance import great_circle
#!pip install shapely
from shapely.geometry import Point
!pip install geopandas
import geopandas as gpd
import glob
import branca.colormap as cm
import random
from folium.features import GeoJsonTooltip, GeoJsonPopup

## 1. API 불러오기_1번만 작동

In [2]:
# 국립중앙의료원_전국 병·의원 찾기 서비스 인증키 불러오기
url_H = 'http://apis.data.go.kr/B552657/HsptlAsembySearchService/getHsptlMdcncFullDown'
api_H = 'SOecclsdpfuvGmi+5d/vkz3M5XiRZEpLLLAm6oeRb4lI6TJSEsmP4nOvH6DfEznSzCw8o3+Ktedgq18jQIGUmg=='

# 국립중앙의료원_전국 약국 정보 조회 서비스 인증키 불러오기
url_M = "http://apis.data.go.kr/B552657/ErmctInsttInfoInqireService/getParmacyFullDown"
api_M = 'SOecclsdpfuvGmi+5d/vkz3M5XiRZEpLLLAm6oeRb4lI6TJSEsmP4nOvH6DfEznSzCw8o3+Ktedgq18jQIGUmg=='

In [3]:
# API에서 totalCount를 자동으로 가져와 전체 데이터를 수집
def fetch_all_data(api_url, api_key, num_of_rows=100, delay=0.05):  # 50ms 대기

    # totalCount 가져오기
    params = {
        'ServiceKey': api_key,
        'pageNo': '1',
        'numOfRows': '1'  # 한 개만 요청하여 totalCount 확인
    }
    response = requests.get(api_url, params=params)
    
    if response.status_code == 200:
        root = ET.fromstring(response.content)
        total_count_elem = root.find(".//totalCount")
        total_count = int(total_count_elem.text) if total_count_elem is not None else 0
    else:
        print(f"totalCount 가져오기 실패: 상태 코드 {response.status_code}")
        return []
    
    all_responses = []
    total_pages = (total_count // num_of_rows) + (1 if total_count % num_of_rows else 0)

    for page in range(1, total_pages + 1):
        params = {
            'ServiceKey': api_key,
            'pageNo': str(page),
            'numOfRows': str(num_of_rows)
        }
        response = requests.get(api_url, params=params)

        if response.status_code == 200:
            all_responses.append(response.content)
        else:
            print(f"페이지 {page} 요청 실패: 상태 코드 {response.status_code}")

        time.sleep(delay)  # 50ms 대기 (30TPS 제한 맞추기)

    return all_responses

# fetch_all_data()로 가져온 XML 데이터를 DataFrame으로 변환
def convert_xml_to_dataframe(xml_data_list):
    
    parsed_data = []

    for xml_data in xml_data_list:
        root = ET.fromstring(xml_data)

        for item in root.findall(".//item"):
            data = {child.tag: child.text for child in item}
            parsed_data.append(data)

    return pd.DataFrame(parsed_data) if parsed_data else None

In [4]:
# fetach_all_data() 적용
response_H = fetch_all_data(url_H, api_H)
response_M = fetch_all_data(url_M, api_M)

# convert_xml_to_dataframe() 적용
df_H = convert_xml_to_dataframe(response_H)
df_M = convert_xml_to_dataframe(response_M)

# 엑셀 파일로 추출
df_H.to_excel("C:/Users/lglg/OneDrive/캡스톤/병원 풀데이터.xlsx", index=False)
df_M.to_excel("C:/Users/lglg/OneDrive/캡스톤/약국 풀데이터.xlsx", index=False)

## 2. 병원 데이터 전처리

In [2]:
# 병원 풀데이터 불러오기
hos = pd.read_excel("C:/Users/lglg/OneDrive/캡스톤/병원 풀데이터.xlsx")
#hos = pd.read_excel("C:/Users/Admin/OneDrive/캡스톤/병원 풀데이터.xlsx")

# 데이터프레임 형태로 변경
hos_df = pd.DataFrame(hos)
print(hos_df.shape)

(77707, 34)


In [3]:
# 병원 풀데이터에서 필요 없는 변수제거
drop_cols = ['dutyEmcls', 'dutyEmclsName', 'dutyEryn', 'dutyInf', 'dutyMapimg',
             'dutyName', 'dutyTel1', 'dutyTime1c', 'dutyTime1s', 'dutyTime2c',
             'dutyTime2s', 'dutyTime3c', 'dutyTime3s', 'dutyTime4c', 'dutyTime4s',
             'dutyTime5c', 'dutyTime5s', 'hpid', 'postCdn1', 'postCdn2',
             'rnum', 'dutyTime6c', 'dutyTime6s', 'dutyTime7c', 'dutyTime7s',
             'dutyTime8c', 'dutyTime8s', 'dutyEtc', 'dutyTel3']

# 필요한 컬럼만 추려서 삭제
drop_cols_exist = [col for col in drop_cols if col in hos_df.columns]
hos_df = hos_df.drop(columns=drop_cols_exist)

# 각 변수 결측값 개수 확인
print(hos_df.isnull().sum())

dutyAddr      0
dutyDiv       0
dutyDivNam    0
wgs84Lat      0
wgs84Lon      0
dtype: int64


In [4]:
# 'dutyAddr' 열의 이름을 'Jibun'으로 변경
hos_df.rename(columns={'dutyAddr': 'Jibun'}, inplace=True)

### 시도 매핑

In [5]:
# hos_df 복사
hos_df_sido = hos_df.copy()

# 시도 매핑 딕셔너리
sido_mapping = {
    '서울': '서울특별시', '부산': '부산광역시', '대구': '대구광역시',
    '인천': '인천광역시', '광주': '광주광역시', '대전': '대전광역시',
    '울산': '울산광역시', '세종': '세종특별자치시', '경기': '경기도',
    '강원': '강원특별자치도', '충북': '충청북도', '충남': '충청남도',
    '전북': '전북특별자치도', '전남': '전라남도', '경북': '경상북도',
    '경남': '경상남도', '제주': '제주특별자치도', '전라북도': '전북특별자치도'
}

# 시도 추출 사용자 정의 함수 생성
def extract_sido(addr):
    if pd.isnull(addr):
        return None

    # 전체 시도명(풀네임)을 먼저 확인
    for full_sido in sido_mapping.values():
        if addr.startswith(full_sido):
            return full_sido

    # 시도 약어 매핑
    for short, full in sido_mapping.items():
        if addr.startswith(short):
            return full

    return None  # 매칭되지 않는 경우

# 위치 정의 함수 생성
def move_column_after(df, target_col, after_col):

    cols = df.columns.tolist()

    if target_col not in cols or after_col not in cols:
        raise ValueError(f"'{target_col}' 또는 '{after_col}' 컬럼이 존재하지 않습니다.")

    cols.remove(target_col)
    idx = cols.index(after_col)
    cols.insert(idx + 1, target_col)
    return df[cols]

# 'Jibun'에서 'Sido' 추출
hos_df_sido['Sido'] = hos_df_sido['Jibun'].apply(extract_sido)

# 'Sido'를 'Jibun' 다음으로 이동
hos_df_sido = move_column_after(hos_df_sido, 'Sido', 'Jibun')

# 각 변수 결측값 개수 확인
print(hos_df_sido.isnull().sum())

Jibun         0
Sido          0
dutyDiv       0
dutyDivNam    0
wgs84Lat      0
wgs84Lon      0
dtype: int64


In [6]:
sido_list = hos_df_sido['Sido'].dropna().unique().tolist()
print(sorted(sido_list))

['강원특별자치도', '경기도', '경상남도', '경상북도', '광주광역시', '대구광역시', '대전광역시', '부산광역시', '서울특별시', '세종특별자치시', '울산광역시', '인천광역시', '전라남도', '전북특별자치도', '제주특별자치도', '충청남도', '충청북도']


### 시군구 매핑

In [7]:
# 전국 법정동 파일 불러오기
all_df = pd.read_csv("C:/Users/lglg/OneDrive/캡스톤/국토교통부_전국 법정동_20250415.csv",encoding='utf-8')
sigungu_list = all_df['시군구명'].dropna().unique().tolist()

# 시군구 추출 함수 정의
def extract_sigungu(addr, sigungu_list):
    if pd.isnull(addr):
        return None
    words = re.findall(r'\w+', addr)  # 단어 단위 추출
    for word in words:
        word = re.sub(r'[^\w가-힣]', '', word)  # 특수문자 제거
        if word in sigungu_list:
            return word
    return None

# hos_df_sido 복사
hos_df_sigungu = hos_df_sido.copy()

# 데이터프레임에서 'Jibun' 열을 기준으로 시군구 추출
hos_df_sigungu['Sigungu'] = hos_df_sigungu['Jibun'].apply(lambda x: extract_sigungu(x, sigungu_list))

# 'Sigungu'를 'Sido' 다음으로 이동
hos_df_sigungu = move_column_after(hos_df_sigungu, 'Sigungu', 'Sido')

# 'Sigungu'가 결측이고 'Sido'가 '세종특별자치시'인 행만 선택하여 보완
mask = (hos_df_sigungu['Sigungu'].isnull()) & (hos_df_sigungu['Sido'] == '세종특별자치시')
hos_df_sigungu.loc[mask, 'Sigungu'] = '세종시'

# 결과 확인
print(hos_df_sigungu['Sigungu'].isnull().sum())

8


In [8]:
# 병원 데이터의 결측값 확인해보기
a = hos_df_sigungu[hos_df_sigungu['Sigungu'].isnull()]
print(a['Jibun'].head(10))

16777               경기도 금빛로 24-22 (굿닥터스빌딩 5층)
20152    부산광역시 신암로 34, 108동 202호 (범천경남아파트 상가)
21317                          부산광역시 중앙대로 786
44307            강원특별자치도  금강로 72, 3,5층 (흥만빌딩)
52374             경기도 하이파크3로 84, 503호 (신성프라자)
52542                 경기도 금빛로 27 (광장프라자 403호)
52978                 경기도 중앙로 46 (판타지아 빌딩 4층)
56893          대전광역시  진잠로 164 (지하1층, 지상2층~7층)
Name: Jibun, dtype: object


In [9]:
# 시군구가 결측인 행만 추출
missing_sigungu_df = hos_df_sigungu[hos_df_sigungu['Sigungu'].isnull()].copy()

# 카카오 Map에서 발급받은 REST API KEY
KAKAO_API_KEY = "9c32bb7527635581ea8a635372d899e1"

# 역지오코딩: 위경도 → 주소
def reverse_geocode(lat, lon, api_key):
    url = "https://dapi.kakao.com/v2/local/geo/coord2address.json"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    params = {"x": lon, "y": lat}
    try:
        res = requests.get(url, headers=headers, params=params)
        res.raise_for_status()
        result = res.json()
        return result['documents'][0]['address']['address_name']
    except:
        return None

# 위경도로부터 주소 추출
missing_sigungu_df['역지오코딩_주소'] = missing_sigungu_df.apply(
    lambda row: reverse_geocode(row['wgs84Lat'], row['wgs84Lon'], KAKAO_API_KEY),
    axis=1
)

# 역지오코딩된 주소로부터 시군구 재추출
missing_sigungu_df['시군구_보완'] = missing_sigungu_df['역지오코딩_주소'].apply(
    lambda x: extract_sigungu(x, sigungu_list)
)

# index 기준으로 원본에 시군구 보완 적용
hos_df_sigungu.loc[missing_sigungu_df.index, 'Sigungu'] = missing_sigungu_df['시군구_보완']

# 최종 결측값 확인
print(hos_df_sigungu['Sigungu'].isnull().sum())

0


#### 시도는 다르지만 시군구 명이 같을때

In [10]:
# hos_df_sigungu 복사
hos_df_sigungu_1 = hos_df_sigungu.copy()

# 시도 축약 매핑 정의
sido_abbr = {
    '서울특별시': '서울', '부산광역시': '부산', '대구광역시': '대구',
    '인천광역시': '인천', '광주광역시': '광주', '대전광역시': '대전',
    '울산광역시': '울산', '세종특별자치시': '세종', '경기도': '경기',
    '강원특별자치도': '강원', '충청북도': '충북', '충청남도': '충남',
    '전북특별자치도': '전북', '전라남도': '전남', '경상북도': '경북',
    '경상남도': '경남', '제주특별자치도': '제주'
}

# 시군구별 중복 확인 (시군구만 기준으로)
duplicate_sgg = hos_df_sigungu_1.groupby('Sigungu')['Sido'].nunique()
duplicated_sgg_names = duplicate_sgg[duplicate_sgg > 1].index.tolist()

# 시군구 보완 함수 정의
def revise_sigungu(row):
    if pd.isnull(row['Sido']) or pd.isnull(row['Sigungu']):
        return row['Sigungu']
    
    # 중복되는 시군구 이름만 보완
    if row['Sigungu'] in duplicated_sgg_names:
        short_sido = sido_abbr.get(row['Sido'], row['Sido'])
        return f"{short_sido} {row['Sigungu']}"
    else:
        return row['Sigungu']

# 적용
hos_df_sigungu_1['Sigungu'] = hos_df_sigungu_1.apply(revise_sigungu, axis=1)

# 결과 확인
print(hos_df_sigungu_1.isnull().sum())

Jibun         0
Sido          0
Sigungu       0
dutyDiv       0
dutyDivNam    0
wgs84Lat      0
wgs84Lon      0
dtype: int64


### 읍면동 매핑

In [11]:
# hos_df_sigungu_1 복사
hos_df_eupmywondong = hos_df_sigungu_1.copy()

eup_list = all_df['읍면동명'].dropna().unique().tolist()

# 주소에서 읍면동 추출 함수
def extract_eupmyeondong(addr, eup_list):
    if pd.isnull(addr):
        return None
    words = re.findall(r'\w+', addr)
    for word in words:
        word = re.sub(r'[^\w가-힣]', '', word)
        if word in eup_list:
            return word
    return None

# 'Jibun'에서 'Eupmywondong' 추출
hos_df_eupmywondong['Eupmywondong'] = hos_df_eupmywondong['Jibun'].apply(lambda x: extract_eupmyeondong(x, eup_list))

# 'Eupmywondong'을 'Sigungu' 다음으로 이동
hos_df_eupmywondong = move_column_after(hos_df_eupmywondong, 'Eupmywondong', 'Sigungu')

# 결과 확인
print(hos_df_eupmywondong['Eupmywondong'].isnull().sum())

1081


In [12]:
# 읍면동 결측 대상만 추출
missing_eup_df = hos_df_eupmywondong[
    hos_df_eupmywondong['Eupmywondong'].isnull() & 
    hos_df_eupmywondong['wgs84Lat'].notnull() & 
    hos_df_eupmywondong['wgs84Lon'].notnull()
].copy()

#역지오코딩 주소 가져오기
missing_eup_df['역지오코딩_주소'] = missing_eup_df.apply(
    lambda row: reverse_geocode(row['wgs84Lat'], row['wgs84Lon'], KAKAO_API_KEY),
    axis=1
)

# 역지오코딩된 주소로부터 읍면동 보완 (eup_list 함께 넘기기)
missing_eup_df['읍면동_보완'] = missing_eup_df['역지오코딩_주소'].apply(lambda x: extract_eupmyeondong(x, eup_list))

# 원본 hos_df에 읍면동 보완 업데이트
hos_df_eupmywondong.loc[missing_eup_df.index, 'Eupmywondong'] = missing_eup_df['읍면동_보완']

# 결과 확인
print(hos_df_eupmywondong['Eupmywondong'].isnull().sum())

0


### Hang = Sido + Sigungu + Eupmywondong

In [13]:
# hos_df_eupmywondong 복사
hos_df_hang = hos_df_eupmywondong.copy()

# Sido, Sigungu, Eupmywondong을 혼합한 Hang 변수 생성 함수 정의
def update_hang(df, sido_col='Sido', sigungu_col='Sigungu', eup_col='Eupmywondong', hang_col='Hang'):
    df[hang_col] = df[sido_col].fillna('') + ' ' + df[sigungu_col].fillna('') + ' ' + df[eup_col].fillna('')
    df[hang_col] = df[hang_col].str.replace(r'\s+', ' ', regex=True).str.strip()
    return df

# 적용
hos_df_hang = update_hang(hos_df_hang)

# 'Hang'을 'Jibun' 다음으로 이동
hos_df_hang = move_column_after(hos_df_hang, 'Hang', 'Jibun')

## 3. 약국 데이터 전처리

In [14]:
# 약국 풀데이터 불러오기
ph = pd.read_excel("C:/Users/lglg/OneDrive/캡스톤/약국 풀데이터.xlsx")
#ph = pd.read_excel("C:/Users/Admin/OneDrive/캡스톤/약국 풀데이터.xlsx")

# 데이터프레임 형태로 변경
ph_df = pd.DataFrame(ph)
print(ph_df.shape)

(24938, 29)


In [15]:
# 약국 풀데이터에서 필요 없는 변수제거
drop_cols = ['dutyEtc', 'dutyName', 'dutyFax', 'dutyInf', 'dutyMapimg','dutyTel1', 
             'dutyTime1c','dutyTime2c','dutyTime3c','dutyTime4c','dutyTime5c','dutyTime6c','dutyTime7c','dutyTime8c',
             'dutyTime1s','dutyTime2s','dutyTime3s','dutyTime4s','dutyTime5s','dutyTime6s','dutyTime7s','dutyTime8s',
             'rnum','hpid','postCdn1','postCdn2']

# 필요한 컬럼만 추려서 삭제
drop_cols_exist = [col for col in drop_cols if col in ph_df.columns]
ph_df = ph_df.drop(columns=drop_cols_exist)

# 각 변수 결측값 개수 확인
print(ph_df.isnull().sum())

dutyAddr    0
wgs84Lat    0
wgs84Lon    0
dtype: int64


In [16]:
# 'dutyAddr' 열의 이름을 'Jibun'으로 변경
ph_df.rename(columns={'dutyAddr': 'Jibun'}, inplace=True)

### 시도 매핑

In [17]:
# ph_df 복사
ph_df_sido = ph_df.copy()

# 열 순서 재배치: 'Jibun' 다음에 'Sido'가 오도록
ph_df_sido['Sido'] = ph_df_sido['Jibun'].apply(extract_sido)

# 'Sido'를 'Jibun' 다음으로 이동
ph_df_sido = move_column_after(ph_df_sido, 'Sido', 'Jibun')

# 각 변수 결측값 개수 확인
print(ph_df_sido.isnull().sum())

Jibun       0
Sido        0
wgs84Lat    0
wgs84Lon    0
dtype: int64


In [18]:
# 시도 리스트 확인학
sido_list = ph_df_sido['Sido'].dropna().unique().tolist()
print(sorted(sido_list))

['강원특별자치도', '경기도', '경상남도', '경상북도', '광주광역시', '대구광역시', '대전광역시', '부산광역시', '서울특별시', '세종특별자치시', '울산광역시', '인천광역시', '전라남도', '전북특별자치도', '제주특별자치도', '충청남도', '충청북도']


### 시군구 매핑

In [19]:
# ph_df_sido 복사
ph_df_sigungu = ph_df_sido.copy()

# 데이터프레임에서 'Jibun' 열을 기준으로 시군구 추출
ph_df_sigungu['Sigungu'] = ph_df_sigungu['Jibun'].apply(lambda x: extract_sigungu(x, sigungu_list))

# 'Sigungu'를 'Sido' 다음으로 이동
ph_df_sigungu = move_column_after(ph_df_sigungu, 'Sigungu', 'Sido')

# 'Sigungu'가 결측이고 'Sido'가 '세종특별자치시'인 행만 선택하여 보완
mask = (ph_df_sigungu['Sigungu'].isnull()) & (ph_df_sigungu['Sido'] == '세종특별자치시')
ph_df_sigungu.loc[mask, 'Sigungu'] = '세종시'

# 결과 확인
print(ph_df_sigungu['Sigungu'].isnull().sum())

0


#### 시도는 다르지만 시군구 명이 같을 때

In [20]:
# ph_df_sigungu 복사
ph_df_sigungu_1 = ph_df_sigungu.copy()

# 시군구별 중복 확인 (시군구만 기준으로)
duplicate_sgg = ph_df_sigungu_1.groupby('Sigungu')['Sido'].nunique()
duplicated_sgg_names = duplicate_sgg[duplicate_sgg > 1].index.tolist()

# 적용
ph_df_sigungu_1['Sigungu'] = ph_df_sigungu_1.apply(revise_sigungu, axis=1)

# 결과 확인
print(ph_df_sigungu_1.isnull().sum())

Jibun       0
Sido        0
Sigungu     0
wgs84Lat    0
wgs84Lon    0
dtype: int64


### 읍면동 매핑

In [21]:
# ph_df_sigungu_1 복사
ph_df_eupmywondong = ph_df_sigungu_1.copy()

# 열 순서 재배치: 'Sigungu' 다음에 'Eupmywondong'이 오도록
ph_df_eupmywondong['Eupmywondong'] = ph_df_eupmywondong['Jibun'].apply(lambda x: extract_eupmyeondong(x, eup_list))

# 'Eupmywondong'을 'Sigungu' 다음으로 이동
ph_df_eupmywondong = move_column_after(ph_df_eupmywondong, 'Eupmywondong', 'Sigungu')

# 결과 확인
print(ph_df_eupmywondong['Eupmywondong'].isnull().sum())

2491


In [22]:
# 읍면동 결측 대상만 추출
missing_eup_df = ph_df_eupmywondong[
    ph_df_eupmywondong['Eupmywondong'].isnull() & 
    ph_df_eupmywondong['wgs84Lat'].notnull() & 
    ph_df_eupmywondong['wgs84Lon'].notnull()
].copy()

#역지오코딩 주소 가져오기
missing_eup_df['역지오코딩_주소'] = missing_eup_df.apply(
    lambda row: reverse_geocode(row['wgs84Lat'], row['wgs84Lon'], KAKAO_API_KEY),
    axis=1
)

# 역지오코딩된 주소로부터 읍면동 보완 (eup_list 함께 넘기기)
missing_eup_df['읍면동_보완'] = missing_eup_df['역지오코딩_주소'].apply(lambda x: extract_eupmyeondong(x, eup_list))

# 원본 ph_df에 읍면동 보완 업데이트
ph_df_eupmywondong.loc[missing_eup_df.index, 'Eupmywondong'] = missing_eup_df['읍면동_보완']

# 결과 확인
print(ph_df_eupmywondong['Eupmywondong'].isnull().sum())

0


### Hang = Sido + Sigungu + Eupmywondong

In [23]:
# ph_df_eupmywondong 복사
ph_df_hang = ph_df_eupmywondong.copy()

# 적용
ph_df_hang = update_hang(ph_df_hang)

# 'Hang'을 'Jibun' 다음으로 이동
ph_df_hang = move_column_after(ph_df_hang, 'Hang', 'Jibun')

## 4. 인구 데이터 전처리

In [3]:
# 인구 데이터 불러오기
#pl = pd.read_csv("C:/Users/lglg/OneDrive/캡스톤/행정안전부_지역별(법정동) 성별 연령별 주민등록 인구수_20250430.csv",encoding='cp949')
pl = pd.read_csv("C:/Users/QQQ/Desktop/TeamMG/코드 데이터/캡스톤 코드 전달_0513/캡스톤 코드 전달_0513/사용한 자료/행정안전부_지역별(법정동) 성별 연령별 주민등록 인구수_20250331.csv",encoding='cp949')

# 데이터프레임 형태로 변경
pl_df = pd.DataFrame(pl)
print(pl_df.shape)

(18710, 234)


In [4]:
# 인구 데이터에서 필요없는 변수제거
drop_cols = ['기준연월','리명']
pl_df.drop(columns=drop_cols, inplace=True)

# 각 변수 결측값 개수 확인
print(pl_df.isnull().sum())

법정동코드        0
시도명          0
시군구명         8
읍면동명         0
계            0
            ..
106세여자       0
107세여자       0
108세여자       0
109세여자       0
110세이상 여자    0
Length: 232, dtype: int64


In [5]:
# 시군구명 결측값 확인하기
a = pl_df[pl_df['시군구명'].isnull()]
print(a['시도명'].head(10))

1893    세종특별시
1894    세종특별시
1896    세종특별시
1900    세종특별시
1901    세종특별시
1909    세종특별시
1910    세종특별시
1914    세종특별시
Name: 시도명, dtype: object


In [6]:
# 세종특별자치시인 행의 시군구명을 시도명으로 통일
pl_df.loc[pl_df['시도명'] == '세종특별시', '시군구명'] = '세종시'

# 시군구명 변수 결측값 개수 확인
print(pl_df['시군구명'].isnull().sum())

0


In [7]:
# pl_df 복사
pl_df_jibun = pl_df.copy()

# 시도명, 시군구명, 읍면동명을 혼합한 Jibun 변수 생성
pl_df_jibun['Jibun'] = (
    pl_df_jibun['시도명']+ ' ' +
    pl_df_jibun['시군구명']+ ' ' +
    pl_df_jibun['읍면동명']
).str.strip()

# 'Jibun'을 '법정도코드' 다음으로 이동
pl_df_jibun = move_column_after(pl_df_jibun, 'Jibun', '법정동코드')

# 각 변수 결측값 개수 확인
print(pl_df_jibun.isnull().sum())

NameError: name 'move_column_after' is not defined

### 시도 매핑

In [8]:
# 시도명을 표준화하는 함수 정의
def standardize_sido(sido_raw):
    if pd.isnull(sido_raw):
        return None  # 결측값 처리

    sido_raw = str(sido_raw).strip()  # 공백 제거 및 문자열화

    # 정확히 일치하는 경우 우선 처리
    if sido_raw in sido_mapping.values():
        return sido_raw

    # 축약어로 시작하는 경우 변환
    for short, full in sido_mapping.items():
        if sido_raw.startswith(short):
            return full

    # 변환 불가능한 경우 원본 유지
    return sido_raw

# 표준화 함수 적용
pl_df_jibun['시도명'] = pl_df_jibun['시도명'].apply(standardize_sido)

# 컬럼명 변경
pl_df_jibun.rename(columns={'시도명': 'Sido'}, inplace=True)

# 결과 확인
print("표준화 후 결측값 수:", pl_df_jibun['Sido'].isnull().sum())
print("시도명 목록:", sorted(pl_df_jibun['Sido'].unique()))

NameError: name 'sido_mapping' is not defined

### 시군구 매핑

In [9]:
# pl_df_jibun 복사
pl_df_sigungu = pl_df_jibun.copy()

# '시군구명' 열의 이름을 'Sigungu'으로 변경
pl_df_sigungu.rename(columns={'시군구명': 'Sigungu'}, inplace=True)

# 데이터프레임에서 'Jibun' 열을 기준으로 시군구 추출
pl_df_sigungu['Sigungu'] = pl_df_sigungu['Jibun'].apply(lambda x: extract_sigungu(x, sigungu_list))

# 'Sigungu'를 'Sido' 다음으로 이동
pl_df_sigungu = move_column_after(pl_df_sigungu, 'Sigungu', 'Sido')

# 결과 확인
print(pl_df_sigungu['Sigungu'].isnull().sum())

NameError: name 'extract_sigungu' is not defined

#### 시도는 다르지만 시군구는 다를 때

In [10]:
# 시군구별 중복 확인 (시군구만 기준으로)
duplicate_sgg = pl_df_sigungu.groupby('Sigungu')['Sido'].nunique()
duplicated_sgg_names = duplicate_sgg[duplicate_sgg > 1].index.tolist()

# 적용
pl_df_sigungu['Sigungu'] = pl_df_sigungu.apply(revise_sigungu, axis=1)

# 결과 확인
print(pl_df_sigungu.isnull().sum())

KeyError: 'Column not found: Sido'

### 읍면동 매핑

In [32]:
# pl_df_sigungu 복사
pl_df_eupmywondong = pl_df_sigungu.copy()

# '읍면동명' 열의 이름을 'Eupmywondong'으로 변경
pl_df_eupmywondong.rename(columns={'읍면동명': 'Eupmywondong'}, inplace=True)

# 열 순서 재배치: 'Sigungu' 다음에 'Eupmywondong'이 오도록
pl_df_eupmywondong['Eupmywondong'] = pl_df_eupmywondong['Jibun'].apply(lambda x: extract_eupmyeondong(x, eup_list))

# 'Eupmywondong'를 'Sigungu' 다음으로 이동
pl_df_eupmywondong = move_column_after(pl_df_eupmywondong, 'Eupmywondong', 'Sigungu')

# 결과 확인
print(pl_df_eupmywondong['Eupmywondong'].isnull().sum())

0


### Hang = Sido + Sigungu + Eupmywondong

In [33]:
# pl_df_eupmywondong 복사
pl_df_hang = pl_df_eupmywondong.copy()

# 적용
pl_df_hang = update_hang(pl_df_hang)

# 'Hang'을 'Jibun' 다음으로 이동
pl_df_hang = move_column_after(pl_df_hang, 'Hang', 'Jibun')

### 표준화 처리 확인하기

In [34]:
# 각 데이터 복사
hos_df = hos_df_hang.copy()
ph_df = ph_df_hang.copy()
pl_df = pl_df_hang.copy()

# 시도 표준화가 잘 되었는지 확인하기
print(set(hos_df['Sido']) == set(ph_df['Sido']) == set(pl_df['Sido']))

# 시군구 표준화가 잘 되었는지 확인하기
print(set(hos_df['Sigungu']) == set(ph_df['Sigungu']) == set(pl_df['Sigungu']))

# 읍면동은 너무 많이 존재하기에 생략
# 시군구 표준화가 잘 되었는지 확인하기
print(set(hos_df['Eupmywondong']) == set(ph_df['Eupmywondong']) == set(pl_df['Eupmywondong']))

True
False
False


#### 시군구 표준화 확인 및 처리

In [35]:
# pl_df vs hos_df의 시군구 비교
pl_set = set(pl_df['Sigungu'])
hos_set = set(hos_df['Sigungu'])

print("공통 (pl & hos):", len(pl_set & hos_set))
print("pl에만 있는 시군구:", pl_set - hos_set)
print("hos에만 있는 시군구:", hos_set - pl_set)

공통 (pl & hos): 229
pl에만 있는 시군구: set()
hos에만 있는 시군구: {'인천 남구'}


In [36]:
# 인천의 남구는 미추홀구로 변경됨. 더 확실한 검정을 위해 역지오 코딩 활용
# 인천 남구만 필터링
namgu_df = hos_df[
    (hos_df['Sigungu'] == '인천 남구') &
    (hos_df['wgs84Lat'].notnull()) &
    (hos_df['wgs84Lon'].notnull())
].copy()

namgu_df['역지오코딩_주소'] = namgu_df.apply(
    lambda row: reverse_geocode(row['wgs84Lat'], row['wgs84Lon'], KAKAO_API_KEY),
    axis=1
)

# 시군구명 보완 함수 정의
def extract_sigungu_from_address(addr, sigungu_list):
    if pd.isnull(addr):
        return None
    for sigungu in sigungu_list:
        if sigungu in addr:
            return sigungu
    return None

# 역지오코딩된 주소로부터 시군구 보완
namgu_df['시군구_보완'] = namgu_df['역지오코딩_주소'].apply(lambda x: extract_sigungu_from_address(x, sigungu_list))

# 원본 hos_df에 시군구 보완 업데이트
hos_df.loc[namgu_df.index, 'Sigungu'] = namgu_df['시군구_보완']

# 결과 확인
print(hos_df['Sigungu'].value_counts().get('인천 남구', 0))

# 변경 전후 테이블 확인
print(namgu_df[['역지오코딩_주소', '시군구_보완']].drop_duplicates())

0
                역지오코딩_주소 시군구_보완
44835  인천 미추홀구 주안동 989-1   미추홀구


In [37]:
# pl_df vs ph_df의 시군구 비교
ph_set = set(ph_df['Sigungu'])

print("공통 (pl & ph):", len(pl_set & ph_set))
print("pl에만 있는 시군구:", pl_set - ph_set)
print("ph에만 있는 시군구:", ph_set - pl_set)

공통 (pl & ph): 229
pl에만 있는 시군구: set()
ph에만 있는 시군구: {'인천 남구'}


In [38]:
# 인천 남구만 필터링
namgu_df = ph_df[
    (ph_df['Sigungu'] == '인천 남구') &
    (ph_df['wgs84Lat'].notnull()) &
    (ph_df['wgs84Lon'].notnull())
].copy()

namgu_df['역지오코딩_주소'] = namgu_df.apply(
    lambda row: reverse_geocode(row['wgs84Lat'], row['wgs84Lon'], KAKAO_API_KEY),
    axis=1
)

# 역지오코딩된 주소로부터 시군구 보완
namgu_df['시군구_보완'] = namgu_df['역지오코딩_주소'].apply(lambda x: extract_sigungu_from_address(x, sigungu_list))

# 원본 ph_df에 시군구 보완 업데이트
ph_df.loc[namgu_df.index, 'Sigungu'] = namgu_df['시군구_보완']

# 결과 확인
print(ph_df['Sigungu'].value_counts().get('인천 남구', 0))

# 변경 전후 테이블 확인
print(namgu_df[['역지오코딩_주소', '시군구_보완']].drop_duplicates())

0
                 역지오코딩_주소 시군구_보완
17038  인천 미추홀구 주안동 473-20   미추홀구


In [39]:
# hos_df vs ph_df의 시군구 비교
print("공통 (hos & ph):", len(hos_set & ph_set))
print("hos에만 있는 시군구:", hos_set - ph_set)
print("ph에만 있는 시군구:", ph_set - hos_set)

공통 (hos & ph): 230
hos에만 있는 시군구: set()
ph에만 있는 시군구: set()


In [40]:
# 시군구 표준화 확인하기
print(set(hos_df['Sigungu']) == set(ph_df['Sigungu']) == set(pl_df['Sigungu']))

True


#### 읍면동 표준화 확인 및 처리

In [41]:
# 각 데이터 복사
hos_df = hos_df_hang.copy()
ph_df = ph_df_hang.copy()
pl_df = pl_df_hang.copy()

In [42]:
# hos_df vs pl_df의 읍면동 비교
hos_set = set(hos_df['Eupmywondong'])  # 병원 데이터
pl_set = set(pl_df['Eupmywondong'])    # 인구 데이터

print("공통 읍면동 수 (hos & pl):", len(pl_set & hos_set))

#차집합 추출
only_in_hos = hos_set - pl_set
print("hos_df에만 있는 읍면동:", sorted(only_in_hos))

공통 읍면동 수 (hos & pl): 3106
hos_df에만 있는 읍면동: ['가운동', '고로면', '과해동', '금능동', '금수면', '남사면', '능서면', '동탄면', '모현면', '미아7동', '부동면', '비산5동', '사벌면', '산곡2동', '삼남면', '상계6동', '상봉1동', '세종로', '송림2동', '안석동', '압량면', '양북면', '연희3동', '오포읍', '옥포면', '용산동6가', '용진면', '우리', '유가면', '일광면', '정관면', '중부면', '청량면', '청북면', '충정로1가', '칠원면', '퇴계원면', '하리면', '현풍면', '호명면']


In [43]:
# 해당 행 필터링
hos_only_df = hos_df[hos_df['Eupmywondong'].isin(only_in_hos)]

# 기준 읍면동 리스트 (pl_df 기준)
eup_list = set(pl_df['Eupmywondong'])

# 주소에서 읍면동 추출 함수 정의
def extract_eupmyeondong_from_address(addr, eup_list, original_eup):
    if pd.isnull(addr):
        return original_eup  # 주소도 없으면 원래 값 반환
    for eup in eup_list:
        if eup in addr:
            return eup
    return original_eup  # 주소는 있지만 매칭 실패 → 원래 값 유지

# 좌표 결측 없는 행만 복사
hos_only_geo_df = hos_only_df[hos_only_df['wgs84Lat'].notnull() & hos_only_df['wgs84Lon'].notnull()].copy()

# 역지오코딩 수행
hos_only_geo_df['역지오코딩_주소'] = hos_only_geo_df.apply(
    lambda row: reverse_geocode(row['wgs84Lat'], row['wgs84Lon'], KAKAO_API_KEY),
    axis=1
)

# 읍면동 보완
hos_only_geo_df['Eupmywondong_보완'] = hos_only_geo_df.apply(
    lambda row: extract_eupmyeondong_from_address(row['역지오코딩_주소'], eup_list, row['Eupmywondong']),
    axis=1
)

# 원본에 보완된 읍면동 반영
hos_df.loc[hos_only_geo_df.index, 'Eupmywondong'] = hos_only_geo_df['Eupmywondong_보완']

# 보완 후 결과 확인
print("역지오코딩 후 여전히 hos_df에만 있는 읍면동:",
      sorted(set(hos_df['Eupmywondong']) - set(pl_df['Eupmywondong'])))

역지오코딩 후 여전히 hos_df에만 있는 읍면동: ['과해동', '세종로', '충정로1가']


In [44]:
# 역지오코딩 이후에도 여전히 pl_df에는 존재하지 않는 Eupmywondong 값들을 hos_df에서 삭제
eup_list = set(pl_df['Eupmywondong'].dropna())

# 여전히 hos_df에만 있는 읍면동
only_in_hos_after_fix = set(hos_df['Eupmywondong'].dropna()) - eup_list

# 삭제 전 행 개수
print("삭제 전 행 수:", len(hos_df))

# 삭제
hos_df_drop = hos_df[~hos_df['Eupmywondong'].isin(only_in_hos_after_fix)].copy()

# 삭제 후 행 개수
print("삭제 후 행 수:", len(hos_df_drop))
print("삭제된 읍면동:", sorted(only_in_hos_after_fix))

삭제 전 행 수: 77707
삭제 후 행 수: 77697
삭제된 읍면동: ['과해동', '세종로', '충정로1가']


In [45]:
# ph_df vs pl_df의 읍면동 비교
ph_set = set(ph_df['Eupmywondong'])  # 약국 데이터
pl_set = set(pl_df['Eupmywondong'])    # 인구 데이터

print("공통 읍면동 수 (ph & pl):", len(pl_set & ph_set))

#차집합 추출
only_in_ph = ph_set - pl_set
print("ph_df에만 있는 읍면동:", sorted(only_in_ph))

공통 읍면동 수 (ph & pl): 2356
ph_df에만 있는 읍면동: ['가운동', '과해동', '남사면', '모현면', '부전1동', '삼남면', '세종로', '압량면', '오포읍', '옥포면', '용산동6가', '우리', '유가면', '정관면', '주월1동', '중앙동7가', '청량면', '청북면', '칠원면', '현풍면', '호명면']


In [46]:
# 해당 행 필터링
ph_only_df = ph_df[ph_df['Eupmywondong'].isin(only_in_ph)]

# 좌표 결측 없는 행만 복사
ph_only_geo_df = ph_only_df[ph_only_df['wgs84Lat'].notnull() & ph_only_df['wgs84Lon'].notnull()].copy()

# 역지오코딩 수행
ph_only_geo_df['역지오코딩_주소'] = ph_only_geo_df.apply(
    lambda row: reverse_geocode(row['wgs84Lat'], row['wgs84Lon'], KAKAO_API_KEY),
    axis=1
)

# 읍면동 보완
ph_only_geo_df['Eupmywondong_보완'] = ph_only_geo_df.apply(
    lambda row: extract_eupmyeondong_from_address(row['역지오코딩_주소'], eup_list, row['Eupmywondong']),
    axis=1
)


# 원본에 보완된 읍면동 반영
ph_df.loc[ph_only_geo_df.index, 'Eupmywondong'] = ph_only_geo_df['Eupmywondong_보완']

# 보완 후 결과 확인
print("역지오코딩 후 여전히 ph_df에만 있는 읍면동:",
      sorted(set(ph_df['Eupmywondong']) - set(pl_df['Eupmywondong'])))

역지오코딩 후 여전히 ph_df에만 있는 읍면동: ['과해동', '세종로']


In [47]:
# 역지오코딩 이후에도 여전히 pl_df에는 존재하지 않는 Eupmywondong 값들을 ph_df에서 삭제
# 여전히 ph_df에만 있는 읍면동
only_in_ph_after_fix = set(ph_df['Eupmywondong'].dropna()) - eup_list

# 삭제 전 행 개수
print("삭제 전 행 수:", len(ph_df))

# 삭제
ph_df_drop = ph_df[~ph_df['Eupmywondong'].isin(only_in_ph_after_fix)].copy()

# 삭제 후 행 개수
print("삭제 후 행 수:", len(ph_df_drop))
print("삭제된 읍면동:", sorted(only_in_ph_after_fix))

삭제 전 행 수: 24938
삭제 후 행 수: 24936
삭제된 읍면동: ['과해동', '세종로']


In [48]:
# hos_df_drop에는 있지만 pl_df에는 없는 읍면동
print("hos_df_drop에만 있는 읍면동:", sorted(set(hos_df_drop['Eupmywondong']) - set(pl_df['Eupmywondong'])))

# ph_df_drop에는 있지만 pl_df에는 없는 읍면동
print("ph_df_drop에만 있는 읍면동:", sorted(set(ph_df_drop['Eupmywondong']) - set(pl_df['Eupmywondong'])))

hos_df_drop에만 있는 읍면동: []
ph_df_drop에만 있는 읍면동: []


In [49]:
'''
# Hang 변수 업데이트
hos_df = update_hang(hos_df)
ph_df = update_hang(ph_df)

# 엑셀 파일로 저장
hos_df.to_excel("병원_풀데이터.xlsx", index=False, encoding='utf-8-sig') # 병원 데이터
ph_df.to_excel("약국_풀데이터.xlsx", index=False, encoding='utf-8-sig') # 약국 데이터
pl_df.to_excel("인구_풀데이터.xlsx", index=False, encoding='utf-8-sig') # 인구 데이터
'''

'\n# Hang 변수 업데이트\nhos_df = update_hang(hos_df)\nph_df = update_hang(ph_df)\n\n# 엑셀 파일로 저장\nhos_df.to_excel("병원_풀데이터.xlsx", index=False, encoding=\'utf-8-sig\') # 병원 데이터\nph_df.to_excel("약국_풀데이터.xlsx", index=False, encoding=\'utf-8-sig\') # 약국 데이터\npl_df.to_excel("인구_풀데이터.xlsx", index=False, encoding=\'utf-8-sig\') # 인구 데이터\n'

## 5. Feature engineering_진행중

In [42]:
# 지도에 인구 표시해보기, 히트맵 (folium,히트맵)
# 공간회귀분석...?
# 의료취약지 자동뷴류
# 인구 수 대비 병원,약국 수의 비율 계산

In [14]:
# 각 데이터프레임 읽기
hos_df_f = pd.read_excel("C:/Users/QQQ/Desktop/TeamMG/data/병원_풀데이터.xlsx") # 병원 데이터
ph_df_f = pd.read_excel("C:/Users/QQQ/Desktop/TeamMG/data/약국_풀데이터.xlsx") # 약국 데이터
pl_df_f = pd.read_excel("C:/Users/QQQ/Desktop/TeamMG/data/인구_풀데이터.xlsx") # 인구 데이터

In [21]:
# 서울, 전남, 강원도만 선택하기
ch_sido = ['서울특별시', '전라남도', '강원특별자치도']
hos_filt = hos_df_f[hos_df_f['Sido'].isin(ch_sido)]
ph_filt = ph_df_f[ph_df_f['Sido'].isin(ch_sido)]
pl_filt = pl_df_f[pl_df_f['Sido'].isin(ch_sido)]

# 각 데이터프레임의 형태 확인
print(hos_filt.shape)
print(ph_filt.shape)
print(pl_filt.shape)

(23491, 9)
(6956, 7)
(4454, 231)


### 5-1-1) 시도에 대한 병원 수/약국 비율: 인구 10,000명당 병원, 약국수

In [23]:
# 시도별 병원/약국 수, 인구 수 집계
# 시도별 병원 수 집계
hos_count_by_sido = hos_filt.groupby('Sido').size().reset_index(name='hos_count')

# 시도별 약국 수 집계
ph_count_by_sido = ph_filt.groupby('Sido').size().reset_index(name='ph_count')

# pl_df_f의 '계' 열의 이름을 'Count'로 변경
pl_filt.rename(columns={'계': 'pl_count'}, inplace=True)

# 시도별 인구 수 집계
pop_by_sido = pl_filt.groupby('Sido')['pl_count'].sum().reset_index()

C:\Users\QQQ\AppData\Local\Temp\ipykernel_16592\3329757405.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pl_filt.rename(columns={'계': 'pl_count'}, inplace=True)


In [ ]:
# 병원/약국/인구 데이터 병합
# 병원 + 약국 병합
facility_by_sido = pd.merge(hos_count_by_sido, ph_count_by_sido, on='Sido', how='outer')

# 인구와 병합
merged_by_sido = pd.merge(facility_by_sido, pop_by_sido, on='Sido', how='left')

# 병합데이터 확인
print(merged_by_sido)

,Sido,hos_count,ph_count,pl_count
0,강원특별자치도,1947,702,1512088
1,서울특별시,18950,5417,9331860
2,전라남도,2594,837,1785003


#### 병원 비율 기준으로 내림차순 정렬

In [27]:
# 비율계산: 인구 1000명당 병원/약국 수 rate: 비율
merged_by_sido['hos_rate'] = (merged_by_sido['hos_count'] / merged_by_sido['pl_count'] * 10000).round(2)
merged_by_sido['ph_rate'] = (merged_by_sido['ph_count'] / merged_by_sido['pl_count'] * 10000).round(2)

# 병원 비율 기준으로 내림차순 정렬
result_by_sido = merged_by_sido.sort_values(by='hos_rate', ascending=True)

df_result = result_by_sido[['Sido', 'pl_count', 'hos_count', 'hos_rate']].copy()
df_result

,Sido,pl_count,hos_count,hos_rate
0,강원특별자치도,1512088,1947,12.88
2,전라남도,1785003,2594,14.53
1,서울특별시,9331860,18950,20.31


#### 약국 비율 기준으로 내림차순 정렬

In [28]:
# 약국 비율 기준으로 내림차순 정렬
result_by_sido = merged_by_sido.sort_values(by='ph_rate', ascending=True)

df_result = result_by_sido[['Sido', 'pl_count', 'ph_count', 'ph_rate']].copy()
df_result

,Sido,pl_count,ph_count,ph_rate
0,강원특별자치도,1512088,702,4.64
2,전라남도,1785003,837,4.69
1,서울특별시,9331860,5417,5.80


### 5-1-2) 시군구에 대한 병원 수/약국 비율: 인구 10,000명당 병원, 약국수

In [29]:
# 시도 + 시군구 단위 병원 수 집계
hos_count_by_sigungu = hos_filt.groupby(['Sido', 'Sigungu']).size().reset_index(name='hos_count')

# 시도 + 시군구 단위 약국 수 집계
ph_count_by_sigungu = ph_filt.groupby(['Sido', 'Sigungu']).size().reset_index(name='ph_count')
 
# 시도 + 시군구 단위 인구 수 집계
pop_by_sigungu = pl_filt.groupby(['Sido', 'Sigungu'])['pl_count'].sum().reset_index()

In [33]:
# 병원/약국/인구 데이터 병합
# 병원 + 약국 병합
facility_by_sigungu = pd.merge(hos_count_by_sigungu, ph_count_by_sigungu, on='Sigungu', how='outer')

# 인구와 병합
merged_by_sigungu = pd.merge(facility_by_sigungu, pop_by_sigungu, on='Sigungu', how='left')
print(merged_by_sigungu)

     Sido_x Sigungu  hos_count   Sido_y  ph_count     Sido  pl_count
0   강원특별자치도     강릉시        275  강원특별자치도        95  강원특별자치도    207108
1   강원특별자치도  강원 고성군         32  강원특별자치도        11  강원특별자치도     26810
2   강원특별자치도     동해시        102  강원특별자치도        39  강원특별자치도     86997
3   강원특별자치도     삼척시         67  강원특별자치도        25  강원특별자치도     61485
4   강원특별자치도     속초시        126  강원특별자치도        42  강원특별자치도     80265
..      ...     ...        ...      ...       ...      ...       ...
60     전라남도     장흥군         62     전라남도        16     전라남도     34322
61     전라남도     진도군         53     전라남도        13     전라남도     28358
62     전라남도     함평군         56     전라남도        14     전라남도     29813
63     전라남도     해남군        104     전라남도        33     전라남도     62744
64     전라남도     화순군        105     전라남도        31     전라남도     60600

[65 rows x 7 columns]


#### 병원 비율 기준으로 내림차순 정렬

In [31]:
# 비율계산: 인구 1000명당 병원/약국 수
merged_by_sigungu['hos_rate'] = (merged_by_sigungu['hos_count'] / merged_by_sigungu['pl_count'] * 10000).round(2)
merged_by_sigungu['ph_rate'] = (merged_by_sigungu['ph_count'] / merged_by_sigungu['pl_count'] * 10000).round(2)

# 병원 비율 기준으로 내림차순 정렬
result_by_sigungu = merged_by_sigungu.sort_values(by='hos_rate', ascending=True)

df_result = result_by_sigungu[['Sido', 'Sigungu', 'pl_count', 'hos_count', 'hos_rate']].copy()
df_result.head(10)

,Sido,Sigungu,pl_count,hos_count,hos_rate
9,강원특별자치도,인제군,31131,28,8.99
6,강원특별자치도,양양군,27398,26,9.49
46,전라남도,광양시,155073,159,10.25
7,강원특별자치도,영월군,36442,38,10.43
3,강원특별자치도,삼척시,61485,67,10.90
17,강원특별자치도,횡성군,46010,51,11.08
10,강원특별자치도,정선군,33381,39,11.68
13,강원특별자치도,태백시,37642,44,11.69
5,강원특별자치도,양구군,20510,24,11.70
2,강원특별자치도,동해시,86997,102,11.72


#### 약국 비율 기준으로 내림차순 정렬

In [39]:
# 약국 비율 기준으로 내림차순 정렬
result_by_sigungu = merged_by_sigungu.sort_values(by='ph_rate', ascending=True)

df_result = result_by_sigungu[['Sido', 'Sigungu', 'pl_count', 'ph_count', 'ph_rate']].copy()
df_result.head(10)

,Sido,Sigungu,pl_count,ph_count,ph_rate
123,전라남도,신안군,38835,7,0.18
154,인천광역시,옹진군,19820,4,0.20
159,경상북도,울릉군,9017,2,0.22
146,경상북도,영양군,15281,4,0.26
162,울산광역시,울산 북구,215684,58,0.27
86,부산광역시,부산 강서구,142620,44,0.31
164,울산광역시,울주군,218392,68,0.31
172,경기도,의왕시,153974,47,0.31
151,경상북도,예천군,54214,17,0.31
38,대구광역시,군위군,22428,7,0.31


In [24]:
#추가로 포함하면 좋은 내용
#의료취약지 플래그 변수 추가: 병원 또는 약국이 하나도 존재하지 않는 읍면동은 1, 존재할 경우 0으로 플래깅
#표준화 지표(Z-score)**로 밀도 분포 해석

## 6. 모델링_진행해야 함

## 7. 이미지 시각화_권혁규 코드

### 1) 지도 생성

In [42]:
# 각 데이터 복사
hos_df = hos_df_f.copy() # 병원 데이터
ph_df = ph_df_f.copy() # 약국 데이터
pl_df = pl_df_f.copy() # 인구 데이터

# 병원/약국/보건소에 'type' 열 추가
hos_df['type'] = '병원'
ph_df['type'] = '약국'

# 세로 결합을 위해 필요한 열만 선택
common_cols = ['Jibun', 'Hang', 'Sido', 'Sigungu', 'Eupmywondong', 'wgs84Lat', 'wgs84Lon', 'type']

hos_sel = hos_df[common_cols]
ph_sel = ph_df[common_cols]

# 세로 방향으로 결합
full_df = pd.concat([hos_sel, ph_sel], ignore_index=True)

In [43]:
# 아이콘 설정 함수
def get_icon(category):

    # 병원과 약국에 따라 아이콘 색상과 모양이 다르게 설정
    color = "red" if category == "병원" else "green"
    icon = "medkit" if category == "병원" else "pills"
    return folium.Icon(color=color, icon=icon, prefix='fa')

# 지도 생성 함수
def create_maps_by_region(df, grouped, output_dir='캡스톤 법정동 지도'):
    os.makedirs(output_dir, exist_ok=True)

    # 각 지역 단위로 그룹화된 데이터를 순회
    for region, group in grouped:

        # 위도나 경도가 없는 행 제거
        valid_group = group.dropna(subset=["wgs84Lat", "wgs84Lon"])
        if valid_group.empty:
            continue

        # 지도 초기화: 해당 지역의 중심 좌표로 생성
        m = folium.Map(
            location=[valid_group["wgs84Lat"].mean(), valid_group["wgs84Lon"].mean()],
            zoom_start=13
        )

        # 병원과 약국 각각에 대한 처리
        for category in ['병원', '약국']:
            sub = valid_group[valid_group["type"] == category]
            if sub.empty:
                continue

            # 병원 또는 약국의 위도/경도 배열 추출
            coords = sub[["wgs84Lat", "wgs84Lon"]].to_numpy()

            # DBSCAN 파라미터: 1km 이내의 클러스터 찾기
            epsilon = 1.0 / 6371.0088  # 지구 반지름을 이용한 라디안 거리

            # DBSCAN 클러스터링 (하버사인 거리 기준)
            db = DBSCAN(eps=epsilon, min_samples=2, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
            
            sub = sub.copy()
            sub["cluster"] = db.labels_

            # 각 클러스터에 대해 처리
            for cluster_id in sub["cluster"].unique():
                cluster_points = sub[sub["cluster"] == cluster_id]
                cluster_coords = cluster_points[["wgs84Lat", "wgs84Lon"]].to_numpy()

                # 클러스터 중심 계산
                center_lat = cluster_coords[:, 0].mean()
                center_lon = cluster_coords[:, 1].mean()
                max_dist = max(great_circle((center_lat, center_lon), (lat, lon)).meters for lat, lon in cluster_coords)
                radius = max_dist + 10000

                # 클러스터 중심으로 원 그리기
                folium.Circle(
                    location=(center_lat, center_lon),
                    radius=radius,
                    color='red' if category == '병원' else 'green',
                    weight=2,
                    fill=True,
                    fill_color='red' if category == '병원' else 'green',
                    fill_opacity=0.1
                ).add_to(m)

                # 각 기관 위치에 마커 추가
                for _, row in cluster_points.iterrows():
                    popup_text = f"주소: {row.get('Jibun', 'N/A')}"
                    folium.Marker(
                        location=(row["wgs84Lat"], row["wgs84Lon"]),
                        popup=folium.Popup(popup_text, max_width=300),
                        icon=get_icon(category)
                    ).add_to(m)

        # HTML 파일로 지도 저장
        m.save(os.path.join(output_dir, f"{region}.html"))

    # 전체 지도 HTML 파일들을 ZIP으로 압축
    zip_path = output_dir + ".zip"
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for file in os.listdir(output_dir):
            zipf.write(os.path.join(output_dir, file), arcname=file)

In [44]:
# Sido를 기준으로 지도 생성
create_maps_by_region(full_df, full_df.groupby('Sido'))

FileNotFoundError: [Errno 2] No such file or directory: '캡스톤 법정동 지도\\경기도.html'

### 2) 인구수 시각화

In [49]:
# 법정동 지도 폴더 경로
root_folder = r"C:/Users/QQQ/Desktop/TeamMG/캡스톤 법정동 지도"

# 저장할 폴더 만들기
os.makedirs('C:/Users/QQQ/Desktop/TeamMG/캡스톤 시도별 지도', exist_ok=True)

# 시도별 폴더 목록 가져오기
sido_folders = [f.path for f in os.scandir(root_folder) if f.is_dir()]

In [50]:
# 인구수 시각화를 위한 색상 지정
colormap_list = [
    cm.linear.Blues_09,   # 파랑 계열
    cm.linear.Greens_09,  # 초록 계열
    cm.linear.Purples_09, # 보라 계열
    cm.linear.Reds_09,    # 빨강 계열
    cm.linear.Oranges_09  # 주황 계열
]

# 지도 생성 함수
def create_map(geo_df, output_path):
    
    # 지도 초기화 - 전체 중심 좌표로 지도 생성
    center = geo_df.geometry.centroid.unary_union.centroid
    m = folium.Map(location=[center.y, center.x], zoom_start=7)

    # 시군구별로 반복하여 GeoJson 층 생성
    for sigungu in geo_df['Sigungu'].unique():
        
        # 해당 시군구만 필터링
        subset = geo_df[geo_df['Sigungu'] == sigungu]

        # 인구수가 0인 격자(읍면동)는 제외
        subset = subset[subset['계'] > 0]
        if subset.empty:
            continue  # 해당 시군구에 인구가 없으면 건너뜀

        # 랜덤 색상 맵 선택 후, 해당 시군구의 인구 min/max에 맞춰 색상 스케일 조정
        cmap = random.choice(colormap_list).scale(
            subset['계'].min(), subset['계'].max()
        )

        # 각 읍면동의 인구수에 따라 색상을 지정하는 스타일 함수 정의
        def style_fn(feat, cmap=cmap):
            val = feat['properties']['계']
            return {
                'fillColor': cmap(val),       # 인구수에 따른 색상
                'color': 'black',             # 외곽선 색 (시군구 경계선)
                'weight': 1,                  # 외곽선 두께
                'fillOpacity': 0.7            # 채우기 투명도
            }

        # folium GeoJson 객체로 시각화, 툴팁 및 팝업 포함
        folium.GeoJson(
            subset.to_json(),                # GeoDataFrame을 GeoJSON 형식으로 변환
            style_function=style_fn,        # 스타일 지정
            tooltip=GeoJsonTooltip(         # 마우스 오버 시 툴팁 표시
                fields=['EMD_NM','계','Sigungu'],
                aliases=['Eupmywondong','count','Sigungu'],
                localize=True, sticky=True, labels=True, toLocaleString=True
            ),
            popup=GeoJsonPopup(             # 클릭 시 팝업 표시
                fields=['EMD_NM','계','시군구명'],
                aliases=['Eupmywondong','count','Sigungu'],
                localize=True, labels=True, toLocaleString=True
            )
        ).add_to(m)
    
    # 지도 저장
    m.save(output_path)

In [52]:
# 시도별 지도 생성 및 저장
merged_list = []  # 시도별 병합 데이터를 저장할 리스트

# 시도별 폴더 반복
for sido_folder in sido_folders:
    folder_name = os.path.basename(sido_folder)        # 폴더명 추출 (예: 'LSMD_ADM_SECT_UMD_강원')
    folder_short = folder_name[-2:]                    # 폴더명 끝 2글자 (예: '강원')
    sido_name = sido_mapping.get(folder_short)         # 시도명 매핑 (예: '강원특별자치도' → '강원도')

    # 해당 폴더 내의 shp 파일 찾기
    shp_files = glob.glob(os.path.join(sido_folder, '*.shp'))
    shp_file = shp_files[0]                            # 첫 번째 shp 파일 사용
    geo_df = gpd.read_file(shp_file, encoding='euc-kr')  # 공간 정보 불러오기

    # 인구 데이터에서 해당 시도만 필터링
    pl_sido_df = pl_df[pl_df['Sido'] == sido_name].copy()

    # 읍면동 코드 추출 (법정동코드 앞 8자리)
    pl_sido_df['읍면동코드'] = pl_sido_df['법정동코드'].astype(str).str[:8]

    # 시군구, 읍면동코드 기준으로 인구수 합산
    pl_grouped = pl_sido_df.groupby(['시군구명', '읍면동코드'])['계'].sum().reset_index()

    # 시도명 컬럼 추가
    pl_grouped['시도명'] = sido_name

    # 공간정보(geo_df)와 인구 데이터(pl_grouped) 병합
    merged = geo_df.merge(pl_grouped, left_on='EMD_CD', right_on='읍면동코드')

    # 좌표계를 EPSG:4326 (WGS 84)로 변환 (지도 시각화용)
    merged = merged.to_crs(epsg=4326)

    # 병합된 결과를 리스트에 저장
    merged_list.append(merged)

    # 해당 시도의 인구 지도를 저장
    create_map(merged, f'C:/Users/QQQ/Desktop/TeamMG/캡스톤 시도별 지도/{sido_name}_인구_지도.html')